In [1]:
#import SparkSession
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1574346533459_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fetching Data

In [2]:
#create spar session object
spark=SparkSession.builder.appName('data_minning').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df =spark.read.csv('s3n://tet-project-3/data-csv/join/articles.csv',inferSchema=True,header=True)
df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, _c0: string, id: string, title: string, publication: string, author: string, date: string, year: string, month: string, url: string, content: string]

##  Removing Null values at content

In [5]:
#Removing null values
df.where(col("content").isNull()).show()

from pyspark.sql.functions import when, col, coalesce, array
fill = array().cast("string")
#null_free = when(col("refined_tokens").isNull(), fill).otherwise(col("refined_tokens"))
null_free = coalesce(col("content"), fill)
null_free_df = df.withColumn("null_free", null_free)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+
|                 _c0|                  id|               title|         publication|              author|                date|                year|               month|                 url|content|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+
|                9374|               28110|"UK Jobs Market C...|         Says Green"|           Breitbart|     Jack Montgomery|          2017-02-26|              2017.0|                 2.0|   null|
|               11683|               30419|"Cohn: Trump Beco...|’ ’More Knowledge...|           Breitbart|      Joel B. Pollak|          2017-05-26|              2017.0|                 5.0|   null|
|    

## Data cleaning

In [6]:
from pyspark.ml.feature import RegexTokenizer
tokenization=RegexTokenizer(minTokenLength=2,gaps=False,pattern='[A-Za-z0-9]+',inputCol="null_free",outputCol='tokens')
tokenized_df=tokenization.transform(null_free_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
tokenized_df.select(['tokens']).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
from pyspark.ml.feature import StopWordsRemover
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
refined_df=stopword_removal.transform(tokenized_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
refined_df.select(['tokens','refined_tokens']).show(1,True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|              tokens|      refined_tokens|
+--------------------+--------------------+
|[washington, cong...|[washington, cong...|
+--------------------+--------------------+
only showing top 1 row

## Vectorizing content

In [10]:
# Count Vectorizer
from pyspark.ml.feature import CountVectorizer
count_vec = CountVectorizer(inputCol='refined_tokens',outputCol='features')
cv_df=count_vec.fit(refined_df).transform(refined_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
cv_df.select(['content','refined_tokens','features']).show(10, True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|             content|      refined_tokens|            features|
+--------------------+--------------------+--------------------+
|WASHINGTON  —   C...|[washington, cong...|(259752,[0,1,2,3,...|
|After the bullet ...|[bullet, shells, ...|(259752,[0,2,3,4,...|
|When Walt Disney’...|[walt, disney, ba...|(259752,[0,2,3,4,...|
|Death may be the ...|[death, may, grea...|(259752,[0,2,3,4,...|
|SEOUL, South Kore...|[seoul, south, ko...|(259752,[0,1,4,5,...|
|LONDON  —   Queen...|[london, queen, e...|(259752,[0,4,5,8,...|
|BEIJING  —   Pres...|[beijing, preside...|(259752,[0,1,2,3,...|
|Danny Cahill stoo...|[danny, cahill, s...|(259752,[0,2,3,4,...|
|Just how   is Hil...|[hillary, kerr, f...|(259752,[0,2,3,4,...|
|Angels are everyw...|[angels, everywhe...|(259752,[0,2,3,4,...|
+--------------------+--------------------+--------------------+
only showing top 10 rows

In [ ]:
'''
#target directory 
write_uri='s3n://tet-project-3/data-csv/clean_data/'
#cv_df.coalesce(1).write.format("csv").option("header","true").save(write_uri)
cv_df.write.format("com.databricks.spark.avro").save(write_uri)
'''


## Random Forest for k means in hyperparameters searching

### Elbow method

In [ ]:
# Finding optimal K
from pyspark.ml.clustering import KMeans

import numpy as np
cost = np.zeros(20)

for k in range(2,20):
    print("On progress -> K:",k)
    kmeans = KMeans()\
            .setK(k)\
            .setSeed(1) \
            .setFeaturesCol("features")\
            .setPredictionCol("prediction")

    model = kmeans.fit(cv_df)
    cost[k] = model.computeCost(cv_df) # requires Spark 2.0 or later
    
'''
[
0.0,
0.0,
141618503.0239381,
135548737.71649304,
132498924.75738414,
130001866.08594018,
129669847.53486447,
126992486.45820226,
126005391.3878738,
126674296.42891622,
125201728.46645792,
124607650.1308867,
124337556.3800629,
124049005.8776054,
122377017.12177089,
122218967.42211193,
121157573.45078428,
120665480.17892012,
120456281.04646748,
120378824.04720232,
]

'''

### Silhouette analysis

In [28]:
import time;
import numpy as np

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

def optimal_k(df_in,index_col,k_min, k_max):
    '''
    Determine optimal number of clusters by using Silhoutte Score Analysis.
    :param df_in: the input dataframe
    :param index_col: the name of the index column
    :param k_min: the train dataset
    :param k_min: the minmum number of the clusters
    :param k_max: the maxmum number of the clusters
    :param num_runs: the number of runs for each fixed clusters

    :return k: optimal number of the clusters
    :return silh_lst: Silhouette score
    :return r_table: the running results table

    :author: Wenqiang Feng
    :email:  von198@gmail.com
    '''

    start = time.time()
    
    silh_lst = []
    k_lst = np.arange(k_min, k_max+1)

    for k in k_lst:
        time_per_k = time.time()
        # Trains a k-means model.
        kmeans = KMeans()\
                .setK(k)\
                .setSeed(1)
        model = kmeans.fit(df_in)

        # Make predictions
        predictions = model.transform(df_in)

        # Evaluate clustering by computing Silhouette score
        evaluator = ClusteringEvaluator()
        silhouette = evaluator.evaluate(predictions)
        
        #Timming
        elapsed =  time.time() - time_per_k
        total_time = "%8.0f s."%(elapsed)
        silh_lst.append([k,silhouette,elapsed])
    
    elapsed = time.time() - start
    elapsed =  "%8.0f s."%(elapsed)
    print('+------------------------------------------------------------+')
    print("|         The finding optimal k phase took ", elapsed, "    |")
    print('+------------------------------------------------------------+')


    return silh_lst,elapsed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
silh_lst,elapsed = optimal_k(cv_df,'features',4, 15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+
|         The finding optimal k phase took      3561 s.     |
+------------------------------------------------------------+

In [30]:
print("Total time taken ->", elapsed)
print("K |\t slih\t\t\t |time_used")
for row in silh_lst:
    print(" {} |\t {}t\t\t |{}".format(row[0],row[1],row[2]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total time taken ->     3561 s.
K |	 slih			 |time_used
 4 |	 0.5107027566216764t		 |319.9010844230652
 5 |	 0.5808733148193805t		 |259.93979024887085
 6 |	 0.44831791237608604t		 |262.7337598800659
 7 |	 0.41966227953778296t		 |292.3457410335541
 8 |	 0.4141751880203832t		 |302.30517578125
 9 |	 0.4151641500402796t		 |291.3599603176117
 10 |	 0.4099641432723832t		 |284.6977937221527
 11 |	 0.4074163958964356t		 |267.6127324104309
 12 |	 0.4123863121470718t		 |281.12498784065247
 13 |	 0.40495262521199094t		 |311.3715479373932
 14 |	 0.36444407461420714t		 |342.0701467990875
 15 |	 0.3886007827936504t		 |345.98229789733887

In [31]:
silh_lst

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[4, 0.5107027566216764, 319.9010844230652], [5, 0.5808733148193805, 259.93979024887085], [6, 0.44831791237608604, 262.7337598800659], [7, 0.41966227953778296, 292.3457410335541], [8, 0.4141751880203832, 302.30517578125], [9, 0.4151641500402796, 291.3599603176117], [10, 0.4099641432723832, 284.6977937221527], [11, 0.4074163958964356, 267.6127324104309], [12, 0.4123863121470718, 281.12498784065247], [13, 0.40495262521199094, 311.3715479373932], [14, 0.36444407461420714, 342.0701467990875], [15, 0.3886007827936504, 345.98229789733887]]

## Kmeans functions 

In [55]:
from pyspark.ml.clustering import KMeans
def generate_kMeans_model (k, dt_intern):
    kmeans = KMeans()\
            .setK(k)\
            .setSeed(1) \
            .setFeaturesCol("features")\
            .setPredictionCol("prediction")

    model = kmeans.fit(dt_intern)
    return model.transform(dt_intern)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def kmean_df_analitic(df_kmean):
    grouped_df = df_kmean.groupBy('prediction').count()
    total = df_kmean.count()
    
    porcent_udf = udf(lambda row_count: '{:.3f}%'.format((row_count / total) * 100 ), StringType())
    return grouped_df.withColumn('Data representation', porcent_udf(grouped_df['count']))
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Silhouette tells us to do it with 5

In [56]:
# The optimal is 5
k = 5
kmeans_df = generate_kMeans_model(k,cv_df)
kmeans_df.select(['content','refined_tokens','features','prediction']).show(k, True) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+----------+
|             content|      refined_tokens|            features|prediction|
+--------------------+--------------------+--------------------+----------+
|WASHINGTON  —   C...|[washington, cong...|(259752,[0,1,2,3,...|         4|
|After the bullet ...|[bullet, shells, ...|(259752,[0,2,3,4,...|         3|
|When Walt Disney’...|[walt, disney, ba...|(259752,[0,2,3,4,...|         4|
|Death may be the ...|[death, may, grea...|(259752,[0,2,3,4,...|         4|
|SEOUL, South Kore...|[seoul, south, ko...|(259752,[0,1,4,5,...|         4|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [68]:
df_grouped = kmean_df_analitic(kmeans_df)
df_grouped.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------------+
|prediction| count|Data representation|
+----------+------+-------------------+
|         1|    18|             0.010%|
|         3|  1734|             0.936%|
|         4| 39743|            21.461%|
|         2|    70|             0.038%|
|         0|143621|            77.555%|
+----------+------+-------------------+

### Elbow with 9

In [69]:
# My Optimal
k = 9
kmeans_df = generate_kMeans_model (k, cv_df)
kmeans_df.select(['content','refined_tokens','features','prediction']).show(50, True) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+----------+
|             content|      refined_tokens|            features|prediction|
+--------------------+--------------------+--------------------+----------+
|WASHINGTON  —   C...|[washington, cong...|(259752,[0,1,2,3,...|         7|
|After the bullet ...|[bullet, shells, ...|(259752,[0,2,3,4,...|         8|
|When Walt Disney’...|[walt, disney, ba...|(259752,[0,2,3,4,...|         4|
|Death may be the ...|[death, may, grea...|(259752,[0,2,3,4,...|         3|
|SEOUL, South Kore...|[seoul, south, ko...|(259752,[0,1,4,5,...|         6|
|LONDON  —   Queen...|[london, queen, e...|(259752,[0,4,5,8,...|         1|
|BEIJING  —   Pres...|[beijing, preside...|(259752,[0,1,2,3,...|         0|
|Danny Cahill stoo...|[danny, cahill, s...|(259752,[0,2,3,4,...|         8|
|Just how   is Hil...|[hillary, kerr, f...|(259752,[0,2,3,4,...|         4|
|Angels are everyw...|[angels, everywhe...|(259752,[0,2,3,4,...|         0|
|With Donald

In [70]:
df_grouped = kmean_df_analitic(kmeans_df)
df_grouped.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------------------+
|prediction| count|Data representation|
+----------+------+-------------------+
|         1|103371|            55.820%|
|         6| 11848|             6.398%|
|         3| 10726|             5.792%|
|         5|   235|             0.127%|
|         4|  1677|             0.906%|
|         8|  1000|             0.540%|
|         7|  7406|             3.999%|
|         2|    26|             0.014%|
|         0| 48897|            26.404%|
+----------+------+-------------------+